In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-02-07 09:36:15.855320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 09:36:16.101548: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-07 09:36:20.674697: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '2015-01-01','2021-03-31')
kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1

kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)
print(kospi_train.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2015-01-05  1921.959961  1921.959961  1903.369995  1915.750000  1915.750000   
2015-01-06  1895.479980  1900.020020  1877.380005  1882.449951  1882.449951   
2015-01-07  1878.900024  1887.599976  1876.270020  1883.829956  1883.829956   
2015-01-08  1895.849976  1907.229980  1891.770020  1904.650024  1904.650024   
2015-01-09  1919.800049  1929.099976  1915.849976  1924.699951  1924.699951   

              Volume   Change1  
Date                            
2015-01-05  313400.0 -0.005549  
2015-01-06  303700.0 -0.017382  
2015-01-07  280500.0  0.000733  
2015-01-08  262200.0  0.011052  
2015-01-09  308800.0  0.010527  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(1531, 7)


In [4]:
kospi_test = fdr.DataReader('KS11', '2021-04-01','2022-10-31')
kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1

kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)
print(kospi_test.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2021-04-02  3104.719971  3121.580078  3101.600098  3112.800049  3112.800049   
2021-04-05  3121.229980  3127.199951  3101.860107  3120.830078  3120.830078   
2021-04-06  3123.800049  3139.290039  3110.189941  3127.080078  3127.080078   
2021-04-07  3129.070068  3146.189941  3125.330078  3137.409912  3137.409912   
2021-04-08  3137.250000  3145.810059  3122.090088  3143.260010  3143.260010   

             Volume   Change1  
Date                           
2021-04-02   792700  0.008227  
2021-04-05  1084800  0.002580  
2021-04-06  1279000  0.002003  
2021-04-07  1984800  0.003303  
2021-04-08  1357300  0.001865  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(388, 7)


In [5]:
kospi_train.index = kospi_train.index.strftime('%Y-%m-%d')
print(kospi_train.index)

kospi_test.index = kospi_test.index.strftime('%Y-%m-%d')
print(kospi_test.index)

Index(['2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09',
       '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16',
       ...
       '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24',
       '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31'],
      dtype='object', name='Date', length=1531)
Index(['2021-04-02', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
       '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15',
       ...
       '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21', '2022-10-24',
       '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-31'],
      dtype='object', name='Date', length=388)


In [6]:
import json
embedding = open('/home/work/nlp/embedding/result/news_summary_128_v2.json', encoding='utf-8')
em_dict = json.load(embedding)

In [7]:
ke_vec_train = []
ke_vec_test = []

for index, row in kospi_train.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Close'])
    ke_vec_train.append(a)

for index, row in kospi_test.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Close'])
    ke_vec_test.append(a)

In [8]:
print(ke_vec_train[0], ke_vec_test[0])

[1915.75, 0.004635863471776247, -0.1988895982503891, -0.10539139807224274, 0.037466149777173996, -0.028444591909646988, 0.08479844778776169, -0.02964376099407673, 0.007250863593071699, 0.20522013306617737, -0.06481444835662842, 0.1779291182756424, 0.03535539284348488, -0.041847046464681625, -0.16426554322242737, 0.0029015683103352785, -0.45696794986724854, -0.117669478058815, -0.28759852051734924, 0.4719439744949341, 0.2610802948474884, -0.4905291497707367, 0.02777247130870819, 0.06812150776386261, 0.1401500403881073, 0.3062049150466919, -0.09731797873973846, 0.0690077692270279, -0.11078614741563797, -0.051850639283657074, 0.09598662704229355, 0.09987270087003708, 0.0409577414393425, 0.08385873585939407, 0.237557515501976, 0.22668509185314178, -0.2083946168422699, -0.21732942759990692, -0.4039594233036041, -0.32057464122772217, -0.07146549224853516, -0.10321042686700821, 0.04196401685476303, -0.2400827258825302, 0.00012017972767353058, 0.20029230415821075, -0.005542295053601265, -0.051

In [9]:
kospi_train_c1 = kospi_train['Change1']
kospi_train_c1 = 100 * kospi_train_c1.values

kospi_test_c1 = kospi_test['Change1']
kospi_test_c1 = 100 * kospi_test_c1.values

In [10]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_test_1, y_test_1 = [], []

In [11]:
for i in range(lookback_window, len(ke_vec_train)-1):
    x_train_1.append(ke_vec_train[i - lookback_window:i])
    y_train_1.append(kospi_train_c1[i])
    
for index in range(len(y_train_1)):
    if y_train_1[index] >= 0:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease' 

In [12]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
print(x_train_1.shape, y_train_1.shape)

(1470, 60, 129) (1470,)


In [13]:
for i in range(lookback_window, len(ke_vec_test)-1):
    x_test_1.append(ke_vec_test[i - lookback_window:i])
    y_test_1.append(kospi_test_c1[i])
    
for index in range(len(y_test_1)):
    if y_test_1[index] >= 0:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'    

In [14]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
print(x_test_1.shape, y_test_1.shape)

(327, 60, 129) (327,)


In [15]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_test_1_e = to_categorical(encoder.transform(y_test_1))

print(y_train_1_e.shape, y_test_1_e.shape)

(1470, 2) (327, 2)


In [22]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16, 32],
        #use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_2 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16, 32],
        #use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_2.summary()
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_3 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=4,
        dilations=[1, 2, 4, 8, 16, 32],
        #use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_6 (TCN)                 (None, 100)               260000    
                                                                 
 dense_6 (Dense)             (None, 2)                 202       
                                                                 
Total params: 260,202
Trainable params: 260,202
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_7 (TCN)                 (None, 100)               382900    
                                                                 
 dense_7 (Dense)             (None, 2)                 202       
                                                                 
Total params: 383,102
Trai

In [23]:
print('Train model_1...')
model_1.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

print('Train model_2...')
model_2.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

print('Train model_3...')
model_3.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

Train model_1...
Epoch 1/50
46/46 [==============================] - 5s 17ms/step - loss: 501.2160 - accuracy: 0.4980
Epoch 2/50
46/46 [==============================] - 1s 20ms/step - loss: 98.3818 - accuracy: 0.4932
Epoch 3/50
46/46 [==============================] - 1s 20ms/step - loss: 106.7601 - accuracy: 0.4803
Epoch 4/50
46/46 [==============================] - 1s 19ms/step - loss: 55.3898 - accuracy: 0.4959
Epoch 5/50
46/46 [==============================] - 1s 18ms/step - loss: 63.6781 - accuracy: 0.5156
Epoch 6/50
46/46 [==============================] - 1s 19ms/step - loss: 30.2871 - accuracy: 0.5197
Epoch 7/50
46/46 [==============================] - 1s 19ms/step - loss: 42.7717 - accuracy: 0.5129
Epoch 8/50
46/46 [==============================] - 1s 20ms/step - loss: 27.0166 - accuracy: 0.5048
Epoch 9/50
46/46 [==============================] - 1s 18ms/step - loss: 17.8255 - accuracy: 0.4980
Epoch 10/50
46/46 [==============================] - 1s 18ms/step - loss: 42.6863

46/46 [==============================] - 1s 20ms/step - loss: 3.1889 - accuracy: 0.5088
Epoch 33/50
46/46 [==============================] - 1s 20ms/step - loss: 1.4758 - accuracy: 0.5537
Epoch 34/50
46/46 [==============================] - 1s 20ms/step - loss: 5.3197 - accuracy: 0.5136
Epoch 35/50
46/46 [==============================] - 1s 19ms/step - loss: 2.6471 - accuracy: 0.5476
Epoch 36/50
46/46 [==============================] - 1s 19ms/step - loss: 1.9386 - accuracy: 0.5197
Epoch 37/50
46/46 [==============================] - 1s 20ms/step - loss: 3.1227 - accuracy: 0.5102
Epoch 38/50
46/46 [==============================] - 1s 19ms/step - loss: 1.7617 - accuracy: 0.5265
Epoch 39/50
46/46 [==============================] - 1s 20ms/step - loss: 2.4407 - accuracy: 0.5435
Epoch 40/50
46/46 [==============================] - 1s 19ms/step - loss: 3.1173 - accuracy: 0.5517
Epoch 41/50
46/46 [==============================] - 1s 21ms/step - loss: 1.1556 - accuracy: 0.5646
Epoch 42/50


In [24]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_2.evaluate(x_test_1, y_test_1_e)[1]
model_3.evaluate(x_test_1, y_test_1_e)[1]

11/11 [==============================] - 1s 8ms/step - loss: 1.5808 - accuracy: 0.4801


0.48012232780456543